# Test with SmartDoc Dataset

In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import glob
from pathlib import Path

import cv2
import numpy as np
import matplotlib.pyplot as plt

import xml.dom.minidom
import xml.etree.ElementTree as ET
from pathlib import Path 

sys.path.insert(0, 'd:/Project/kuka-py/')
runingPath=os.path.abspath("")
parentPath = Path(runingPath).resolve().parents[0]
if(sys.path[0]!=parentPath):
    sys.path.insert(0, parentPath)

from kukalib.cardcrop  import  *
print(getVersionInfo())

{'version': '0.3.5', 'date': datetime.date(2023, 7, 7)}


In [6]:
def eval2Point(p1,p2):
    maxDis=30 # max distance from p1 to p2
    confident=0.0
    l=getLineLength(p1,p2)
    if(l<=maxDis):
        confident=((maxDis-l)/maxDis)
    return confident

def get_confidence_2quard(quadr_true,quadr_pred):
    (tl_pred,tr_pred,br_pred,bl_pred)=quadr_pred
    (tl,tr,br,bl)=quadr_true
    tl_c=(eval2Point(tl_pred,tl))
    tr_c=(eval2Point(tr_pred,tr))
    br_c=(eval2Point(br_pred,br))
    bl_c=(eval2Point(bl_pred,bl))

    mean_c=(tl_c+tr_c+br_c+bl_c)/4
    return mean_c

def getListGroundTruth(xmlfile):
    """
        returns:
            list frame points in order [tl,tr,br,bl]. 
    """
    gtList=[]
    doc = xml.dom.minidom.parse(xmlfile)
    frameList = doc.getElementsByTagName("frame")
    for frame in frameList:
        bl=[]
        tl=[]
        tr=[]
        br=[]
        pointList = frame.getElementsByTagName("point")
        for p in pointList:
            name = p.getAttribute("name")
            x= float(p.getAttribute("x"))
            y= float(p.getAttribute("y"))
            if(name=="bl"):
                bl = [x, y]
            elif name == "tl":
                tl =[x,y]
            elif name =="tr":
                tr =[x,y]
            elif name =="br":
                br=[x,y]
        gtList.append([tl,tr,br,bl])
    return gtList
    

def RunTest(dataRootPath,saveRoothPath):
    dataRootPath = os.path.abspath(dataRootPath)
    saveRoothPath = os.path.abspath(saveRoothPath)

    confidenceList=[] 
    showDebug=True
    testResult =[]
    specific_file=""
    if(os.path.isfile(dataRootPath)):
        specific_file=dataRootPath
        dataRootPath=os.path.dirname(dataRootPath)
    for f in glob.glob(os.path.join(dataRootPath,"**"),recursive=True):
        if(os.path.isdir(f) or not f.endswith(".avi")):
            continue
        if(specific_file!="" and specific_file!=f):
            continue

        videoFile = f
        videoName =Path(videoFile).name
        gtfile = videoFile.replace(".avi",".gt.xml")
                           
        savedImagePath=os.path.dirname(f).replace(dataRootPath,saveRoothPath)
        savedImagePath = os.path.join(savedImagePath,videoName)

        if(not os.path.exists(savedImagePath)):
            os.makedirs(savedImagePath)

        gtList =getListGroundTruth(gtfile)
        gtList = np.asarray( gtList)

        video = cv2.VideoCapture(videoFile)
        

        # Check if camera opened successfully
        if (video.isOpened()== False): 
            print("Error opening video stream or file")
        
        # Read until video is completed
        numdebug=0
        
        frame_width = int(video.get(3))
        frame_height = int(video.get(4))
        # Define the codec and create VideoWriter object.The output is stored in 'outpy.avi' file.
        out = cv2.VideoWriter(r'D:/test/'+videoName,cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width,frame_height))

        frameIdx=0
        
        isOpened = video.isOpened()
        print ("Video file: ",f)
        confidenceListByFile =[]
        while(isOpened):
            # Capture frame-by-frame
            ret, frame = video.read()
            if ret == True:
                numdebug +=1
                frameIdx +=1
            
                #print("Processing frame: ", frameIdx,end='')
                cropedImg,debugImg,(topleftPoint,toprightPoint,bottomrightPoint,bottomleftPoint),hasCropped=cropDocument(frame)
                (tl,tr,br,bl)=gtList[frameIdx-1]
                #calculate metric IoU
                confidence = 0
                
                if(hasCropped):
                    confidence=get_confidence_2quard((tl,tr,br,bl),(topleftPoint,toprightPoint,bottomrightPoint,bottomleftPoint))
                #print("-->Done \t Confidence: {:4f}".format(confidence))
                
                isLowConfidence = confidence<0.8
                confidenceList.append(confidence)
                confidenceListByFile.append(confidence)

                # # write out debug
                if(showDebug):
                    croppedImageFile =os.path.join(savedImagePath , "frame_{}.jpg".format(frameIdx))
                    debugImgFile =os.path.join(savedImagePath , "debug_frame_{}.jpg".format(frameIdx))
                    cv2.imwrite(debugImgFile,debugImg)
                    cv2.imwrite(croppedImageFile,cropedImg)

                if(hasCropped and  isLowConfidence ):
                    lowConfidenceFile =os.path.join(savedImagePath,"LowConfidence_frame_{}.jpg".format(frameIdx))
                    cv2.imwrite(lowConfidenceFile,debugImg)

                if(not hasCropped):
                    uncropFile =os.path.join(savedImagePath, "UnCrop_frame_{}.jpg".format(frameIdx))
                    cv2.imwrite(uncropFile,frame)
                
                
            # Break the loop
            else: 
                break
        
        # When everything done, release the video capture object
        video.release()
        out.release()
        print('File {} - Total Frame {} - Confidence {:4f} '.format(f,len(confidenceListByFile),np.mean(confidenceListByFile)))

    print('Done!')
    print('-------------------------------------------')
    print("Total test:",len(confidenceList))
    print('Confidence mean:',np.mean(confidenceList))


dataRootPath =r"d:\ImageData\SmartDoc\background01\datasheet001.avi"
saveRoothPath = r"d:\test\SmartDoc"

RunTest(dataRootPath,saveRoothPath)



Video file:  d:\ImageData\SmartDoc\background01\datasheet001.avi


KeyboardInterrupt: 